In [43]:
import json
from objectbox import (
    Entity,
    Id,
    String,
    Store,
    Box,
    Float32Vector,
    HnswIndex,
    VectorDistanceType,
)
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from pathlib import Path

In [44]:
rules_chunks_pth = Path("../data/podzemelja_chunks.cleaned.json")
rules_chunks: list[dict] = json.loads(rules_chunks_pth.read_text())

In [45]:
rules_chunks[0]

{'id': 'subtitle',
 'content': 'ПОДЗЕМЕЛЬЯ ПЁСИКИ\n\nПравила игры',
 'type': 'subtitle',
 'section': 'main_title',
 'merged_from': 'title',
 'isHeader': False}

In [46]:
for rule in rules_chunks:
    tags = " ".join(
        [
            "#game:Подземелье_и_Песики",
            f"#section:{rule['section']}",
            f"#type:{rule['type']}",
        ]
    )
    rule["content"] = f"{tags}\n---\n{rule["content"]}"

In [47]:
print(rules_chunks[2]["content"])

#game:Подземелье_и_Песики #section:introduction #type:narrative
---
В волшебном королевстве каждый год проходят соревнования по побегу из подземелья. Самые хорошие и смелые пёсики спускаются в глубокие мрачные комнаты, чтобы сразиться с чудовищами и собрать как можно больше трофеев. Победителей ждут почести и слава, титулы и горы золота, а также очень вкусные косточки.


In [48]:
@Entity()
class Rule:
    id = Id
    internal_id = String
    content = String
    section = String
    game = String
    vector = Float32Vector(
        index=HnswIndex(dimensions=768, distance_type=VectorDistanceType.EUCLIDEAN)
    )

In [49]:
store = Store(directory="../db")
rules_box = Box(store=store, entity=Rule)

---

In [51]:
model = SentenceTransformer("../model").to("cpu")

In [52]:
box_objects = [
    Rule(
        internal_id=obj["id"],
        content=obj["content"],
        section=obj["section"],
        game="Подземелье и пёсики",
        vector=model.encode(obj["content"]).tolist(),
    )
    for obj in tqdm(rules_chunks)
]

100%|██████████| 51/51 [00:11<00:00,  4.52it/s]


In [53]:
rules_box.put(box_objects)

In [54]:
rules_box.get(1)

In [55]:
store.close()